# Connections Lab

### Introduction

In this lesson, we'll practice using airflow to connect to our AWS services.  Let's get started.

### Connecting to RDS

Now for this task, let's create a new dag that performs two tasks:

1. Create a task called `five_venues` that queries and logs the first five venues
2. Create a task called `three_categories` that queries and log the first three categories

To accomplish this, we can connect to an RDS instance already set up.  Here is some information you'll need to connect to the database.

In [1]:
host = "foursquare-flask-api.cbdkozm37vkd.us-east-1.rds.amazonaws.com"
database = 'postgres'
password = 'password1'

Ok, now let's create the dag and the two tasks, and use a PostgresHook to produce the requested information in the task logs.  If things are working properly, we can begin by seeing that the tasks have successfully performed.

<img src="./perform_queries.png" width="80%">

And then we can check the logs.

> So first we can see the values from `get_venues` returned.

<img src="./get_venues.png" width="100%">

> And then in the `get_categories` task, we can see that the values of Pizza, Italian, and Bar were returned.

<img src="./get_categories.png" width="90%">

Great, so we were successfully able to connect to our instances.

### Summary

In this lesson, we practiced working with and setting up connections so that we could connect to our RDS database.  In future lessons, we'll see how we can use the connections to perform our steps of loading data RDS to S3 and S3 to redshift.